# Generating the Mandelbrot and Julia Sets

#### Written by: David Sutherland - Group C

### Contents

1. [Introduction](#Introduction)

2. [System Generator Architecture](#System-Generator-Architecture)

3. [Mandelbrot Set Jupyter Implementation](#Mandelbrot-Set-Jupyter-Implementation)

4. [Julia Set Jupyter Implementation](#Julia-Set-Jupyter-Implementation)

5. [References](#References)

### Introduction

The Mandelbrot and Julia sets are two of the most famous fractals. They both revolve around iterating the equation $ z_{n+1} = z_{n}^{2} + c $. If the value of $z_{n+1}$ tends to infinity then the chosen point is not in the set. $^{[1]}$

In the Mandelbrot set $z_{0}$ is always set to 0 and the value for $c$ varies. 

In the Julia set $c$ is set to a constant complex number and $z_{0}$ varies. Different values for $c$ result in different, interesting patterns being produced.

In this notebook both the Mandelbrot and Julia sets will be generated using an IP core on the PYNQ board that has been developed in System Generator and Vivado. 

An explanation of the System Generator architecture will be given as well as Python code.

User interactivity is also enabled which will allow the user to change certain properties for the generation of the Mandelbrot Set and Julia Set.

# System Generator Architecture

The block diagram below shows the architecture used to generate the values for both sets by implementing the equation $ z_{n+1} = z_{n}^{2} + c $. 



![](images/mandelbrotsysgen.PNG)

The system is built this way as $z_{n}$ could be a complex number and so squaring it has to be done differently compared to a real number. The working for finding the square of a complex number is as follows:

$$z = a + jb$$

$$z^{2} = (a + jb)^{2}$$

$$z^{2} = a^{2} + j2ab + j^{2}b^{2}$$ 

$$z^{2} = a^{2} + j2ab - b^{2}$$ 

$$z^{2} = (a^{2} - b^{2}) + j(2ab) $$ 

Then adding a complex number $c = x + jy$ to this gives:

$$ z_{n+1} = z_{n}^{2} + c $$

$$ z_{n+1} = (a^{2} - b^{2}) + j(2ab) + (x+jy) $$

$$ z_{n+1} = (a^{2} - b^{2} + x) + j(2ab + y) $$

This is the equation that has been built in the System Generator model.

The Gateway In blocks are implemented using the AXI4-Lite interface. 

The block diagram below shows the System Generator architecture used to calculate the magnitude of a complex number. This was required to determine if the current value being iterated was bounded or not. 

![](images/magnitudesysgen.PNG)

Initially this was attempted using the sqrt block in System Generator however this was not suitable as it required floating point operations which would take several clock cycles to compute on an FPGA. A magnitude approximation method was then chosen as it meant the operation would run quicker. This method does have a lower accuracy compared to the floating point method however this was not an issue as the answer did not need to be extremely accurate - an approximation would suffice.

The magnitude approximation equation is as follows:
$|z|≈\alpha Max(a,b)+\beta Min(a,b) $

The values for $\alpha$ and $\beta$ are chosen to produce the lowest error. Richard G Lyons proved that the values can be set to the following $^{[2]}$:

$$\alpha=\frac{2cos(\frac{π}{8})}{1+cos(\frac{π}{8})} = 0.9604$$

$$\beta=\frac{2sin(\frac{π}{8})}{1+cos(\frac{π}{8})} = 0.3978$$

The CMult blocks are not exactly these numbers due to not being able to precisely represent these numbers because the input is only 9 fractional bits but this is a close enough approximation for this scenario.

In the System Generator model a relational block is used to determine which number is larger and then two multiplexers are used to correctly multiply the greater and smaller numbers by $\alpha$ and $\beta$ respectively. The two results from this are then added together to obtain the magnitude approximation.

# Mandelbrot Set Jupyter Implementation

This section will detail how to generate the Mandelbrot Set using the IP core developed from the design above. Firstly, the overlay class needs to be imported:

In [ ]:
from pynq import Overlay

Next the overlay needs to be programmed onto the PYNQ-Z2 board and the Tcl file will be parsed to understand the contents of the overlay. A warning may appear but this can be ignored:

In [ ]:
ol = Overlay("assets/mandelbrot_design_wrapper.bit")

Running the next line of code will display a file containing information about the overlay. This file identifies the IP to be called mandelbrot_full_0 in the DefaultIP driver.

In [ ]:
ol?

The IP will be assigned to a more convenient name, 'mandip':

In [ ]:
mandip = ol.mandelbrot_full_0

The next cell defines a function called 'to_signed' which is used to read back negative values from the PYNQ board as Python naturally interprets the read values as unsigned.

In [ ]:
    def to_signed(val,b):
         signedVal = val-(2**b)*int(str((val)>>b-1))
         return signedVal

The next cell defines the Mandelbrot function which uses the IP to find the values which are in the Mandelbrot Set.

Currently it is not possible to write real numbers to the IP so the values are multiplied by $2^{9}$ as a workaround and because the IP uses 9 fractional bits. This is why the System Generator model contains reinterpret blocks as they convert this value back to the actual one to use. When the values are read back they are first changed to signed and then multiplied by $2^{-9}$ to get the correct value.

The magnitude does not need to be changed to signed because it will always be a positive value.

In [ ]:
def mandelbrot(c_real, c_imag, MAX_ITER):
    z_real = 0 #These values are always set to 0 when the function is called
    z_imag = 0
    n = 0
    magnitude = 0
    while magnitude <= 2 and n < MAX_ITER: #MAX_ITER is chosen by user
        mandip.write(0x00,int(z_real*(2**9)))
        mandip.write(0x04,int(z_imag*(2**9))) 
        mandip.write(0x08,int(c_real*(2**9))) 
        mandip.write(0x0C,int(c_imag*(2**9))) 
        z_real = to_signed(int(mandip.read(0x18)),32) * 2**-9 #new zre value for next iteration
        z_imag = to_signed(int(mandip.read(0x14)),32) * 2**-9 #new zim value for next iteration
        magnitude = mandip.read(0x10) * 2**-9 #magnitude output
        n += 1 #Move to next iteration
    return n

Once the next cell is ran you will be able to use a slider to change the maximum number of iterations that the 'mandelbrot' function goes through.

*You do not need to re-run this cell once you have chosen your value*

In [ ]:
from ipywidgets import *
def update(Max_iters):
    
    Max_iters = Max_iters
    
Max_iters = widgets.IntSlider(min = 5, max = 100,step = 1)

interact(update,Max_iters = Max_iters)
print("The higher the maximum number of iterations, the more defined the set will be. Recommended value: 25")

The next cell goes through every pixel in the image and converts it to a complex number. This complex number is used as $c$ and is used in the 'mandelbrot' function to find out if it is in the set. If the maximum number of iterations is reached then that value of $c$ is in the set. The value returned by the function determines the colour of that pixel. The black pixels are the complex numbers that are in the Mandelbrot Set

The length of time to run depends on the size of the image and the maximum number of iterations chosen in the previous cell.

*Code adapted from $^{[3]}$*

In [ ]:
from PIL import Image, ImageDraw #Import library used to draw fractal
MAX_ITER = Max_iters.value #Value taken from Max_iters slider

# Image size (pixels)
WIDTH = 150
HEIGHT = 100

# Plot window
RE_START = -2
RE_END = 1
IM_START = -1
IM_END = 1

im = Image.new('HSV', (WIDTH, HEIGHT), (0, 0, 0)) #Configure image to specified size and use 
draw = ImageDraw.Draw(im)

for x in range(0, WIDTH): #Go through every pixel using nested loop
    for y in range(0, HEIGHT): 
        
        # Convert pixel coordinate to complex number
        creal = RE_START + (x / WIDTH) * (RE_END - RE_START)
        cimag = IM_START + (y / HEIGHT) * (IM_END - IM_START)
        
        m = mandelbrot(creal,cimag, MAX_ITER) # Compute the number of iterations using mandelbrot function
        
        hue = int(255 * m / MAX_ITER) # The colour depends on the number of iterations taken for the number to become unbounded
        saturation = 255
        value = 255 if m < MAX_ITER else 0 #If MAX_ITER is reached then the colour is set to black (In the set)
        # Plot the point
        draw.point([x, y], (hue, saturation, value))

im.convert('RGB').save('images/mandelbrotoutput.PNG', 'PNG') #Save image - you can change this path if you wish

The next cell will display your generated fractal once ran:

In [ ]:
from IPython.display import Image
Image(filename='images/mandelbrotoutput.PNG') #If you changed the path in the previous cell you need to change this one too

If you change the values in the plot window section you will effectively be able to zoom into parts of the set to discover some interesting properties, one being that if you zoom into some of the 'spikes' coming out of the main circle you may actually see the original image again! This means that the Mandelbrot Set is infinite and you could theoretically zoom in forever on the boundaries of the set. Unfortunately zooming in too far in this notebook will result in a pixelated image.

If you chose to run the code with a large image size and high number of maximum iterations you will have discovered that this look quite a long time to run! This is because AXI4-Lite is only capable of single beat transfers meaning only one piece of data can be transferred at a time so it will be slow to process large amounts of data like the Mandelbrot function needs to do. 

# Julia Set Jupyter Implementation 

This section will detail how to generate a Julia Set fractal. Each point in the Mandelbrot Set is associated with it's own Julia Set. 

Since the equation used in the Julia set is the same, the previous IP can be re-used.

Because the IP used is the same we can go straight to defining the Julia set function. This function is slightly different to the Mandelbrot Set function as there are more arguments. 

In [ ]:
def julia(c_real,c_imag,z_real,z_imag, MAX_ITERS):
    magnitude = 0
    n = 0
    while magnitude <= 2 and n < MAX_ITERS:
        mandip.write(0x00,int(z_real*(2**9))) #zre
        mandip.write(0x04,int(z_imag*(2**9))) #zim
        mandip.write(0x08,int(c_real*(2**9))) #cre
        mandip.write(0x0C,int(c_imag*(2**9))) #cim
        z_real = to_signed(int(mandip.read(0x18)),32) * 2**-9 #new zre value for next iteration
        z_imag = to_signed(int(mandip.read(0x14)),32) * 2**-9 #new zim value for next iteration
        magnitude = mandip.read(0x10) * 2**-9 #mag out
        n += 1
    return n 

The following cell will let you choose a value for the complex number $c$. Changing this value will change how the fractal looks. Some values are suggested which can produce some interesting fractals. You can use the arrow keys to choose the desired value when you get near to it.

*You do not need to re-run this cell once you have chosen your values*

In [ ]:
from ipywidgets import *
def update(c_real, c_imag):
    
    c_real = c_real
    c_imag = c_imag
    
c_real = widgets.FloatSlider(min = -2, max = 1,step = 0.01)
c_imag = widgets.FloatSlider(min = -2, max = 1,step = 0.01)

interact(update,c_real = c_real, c_imag=c_imag)

print("Some interesting values to try out: ")
print("c = -1 + j0")
print("c = 0.37 + j0.1")
print("c = -0.4 - j0.59")
print("c = -0.54 + j0.54")

In [ ]:
from PIL import Image, ImageDraw #Import library used to draw fractal

creal = c_real.value #Get complex number values from previous cell
cimag = c_imag.value

# Image size (pixels)
WIDTH = 150
HEIGHT = 100

# Plot window - reduce these values to zoom in
RE_START = -2
RE_END = 2
IM_START = -1.2
IM_END = 1.2

MAX_ITER = 55 #max number of iterations used in julia function - feel free to change this

im = Image.new('HSV', (WIDTH, HEIGHT), (0, 0, 0)) 
draw = ImageDraw.Draw(im)

for x in range(0, WIDTH): #nested loop to go through every pixel in image
    for y in range(0, HEIGHT):
        
        # Convert pixel coordinate to complex number
        z0real = RE_START + (x / WIDTH) * (RE_END - RE_START)
        z0imag = IM_START + (y / HEIGHT) * (IM_END - IM_START)
        
        # Compute the number of iterations
        m = julia(creal,cimag,z0real, z0imag, MAX_ITER)
        
        hue = int(255*(m/MAX_ITER)) #colour of pixel depends on value of m - the number of iterations needed to become unbound
        saturation = 255
        value = 255 if m < MAX_ITER else 0

        draw.point([x, y], (hue, saturation, value)) #set current pixel to the colour determined above

im.convert('RGB').save('images/juliaoutput.PNG', 'PNG') #save image - you can change this path if you wish

Run the next cell to display the generated Julia Set fractal:

In [ ]:
from IPython.display import Image
Image(filename='images/juliaoutput.PNG') #if you changed the path in the previous cell you will need to change this one too

Be sure to experiment with changing the value for $c$ to find interesting designs.

------------

From reading the code you can see how the generation of the Julia Set differs from the Mandelbrot Set. In the Mandelbrot Set $z_{0}$ is always 0 and $c$ is the complex number to represent a pixel while in the Julia Set $z_{0}$ is the complex number converted from the current pixel and $c$ is a constant value.

I hope you enjoyed going through this notebook and creating some interesting fractals!

# References

[1] http://www.alunw.freeuk.com/mandelbrotroom.html

[2] Lyons, R., 2011. Understanding Digital Signal Processing. 2nd ed. Prentice Hall, pp.480-482.

[3] https://www.codingame.com/playgrounds/2358/how-to-plot-the-mandelbrot-set/adding-some-colors